In [1]:
import os
import yaml
import re

DOCS_DIR = "docs"

# 👇 在这里配置要隐藏的文件夹名称（仅目录名，不含路径）
HIDDEN_DIRS = {
    ".git",
    ".github",
    "hidden_folder",
    "Archive",
    "__pycache__"
}

def extract_title_from_file(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            # 匹配 Markdown 一级标题
            match = re.match(r"# (.+)", line)
            if match:
                return match.group(1).strip()
    return os.path.splitext(os.path.basename(filepath))[0].replace("_", " ").title()

def numeric_sort_key(name):
    match = re.search(r"(\d+)", name)
    return int(match.group(1)) if match else float('inf')

def generate_nav(path=DOCS_DIR):
    nav = []

    for root, dirs, files in os.walk(path):
        rel_dir = os.path.relpath(root, path)
        if rel_dir == ".":
            rel_dir = ""

        # 排除隐藏目录
        dirs[:] = [d for d in sorted(dirs, key=numeric_sort_key) if d not in HIDDEN_DIRS and not d.startswith(".")]

        items = []
        for file in sorted(files):
            if file.endswith(".md"):
                filepath = os.path.join(root, file)
                nav_title = extract_title_from_file(filepath)
                relative_path = os.path.relpath(filepath, path).replace("\\", "/")
                items.append({nav_title: relative_path})

        if rel_dir == "":
            nav.extend(items)
        elif items:
            section_title = rel_dir.replace("_", " ").title()
            nav.append({section_title: items})

    return nav

# 加载并更新 mkdocs.yml
with open("mkdocs.yml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

config["nav"] = generate_nav()

with open("mkdocs.yml", "w", encoding="utf-8") as f:
    yaml.dump(config, f, allow_unicode=True, sort_keys=False)

print("✅ nav 自动生成完毕，已根据设置隐藏目录！")


✅ nav 自动生成完毕，已根据设置隐藏目录！
